In [ ]:
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace

In [ ]:
class SnDBot:
    def __init__(self, output_file="emotions.txt"):
        self.cap = None
        self.output_file = output_file
        with open(self.output_file, 'w', encoding='utf-8') as f: f.write("")
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def start_capture(self, source="video.mp4"):
        self.cap = cv2.VideoCapture(source)
        if not self.cap.isOpened():
            raise Exception("Error: Cannot open the camera or video file")

    def detect_emotion(self, frame):
        try:
            emotions = DeepFace.analyze(frame, actions=['emotion'])
            if isinstance(emotions, list) and len(emotions) > 0:
                dominant_emotion = max(emotions[0]['emotion'].items(), key=lambda x: x[1])
                return dominant_emotion
        except Exception as e:
            pass
        
        return None
    
    def detect_face(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))
        return faces
    
    def document(self, emotion):
        timestamp = int(self.cap.get(cv2.CAP_PROP_POS_MSEC)) if self.cap is not None else 0
        with open(self.output_file, 'a', encoding='utf-8') as f:
            if emotion: f.write(f"{timestamp} {emotion[0]} {emotion[1]:.2f}\n")
            else: f.write(f"{timestamp} .\n")

    def process_frame(self, frame):
        emotion = None
        face_location = self.detect_face(frame)
        
        if face_location is not None:
            x, y, w, h = face_location[0]
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
            
            face = frame[y:y+h, x:x+w]
            emotion = self.detect_emotion(face)

            if emotion:
                emotion_text = f"{emotion[0]} ({emotion[1]:.2f})"
                cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255, 0), 2)
                bar_length = int(emotion[1]*100)
                #cv2.rectangle(frame, (x, y-30), (x+bar_length, y-20), (0, 255, 0), -1)

        return frame, emotion
    
    def run(self):
        if self.cap is None:
            self.start_capture()
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            processed_frame, current_emotion = self.process_frame(frame)
            
            cv2.imshow('Emotion Detection', processed_frame)

            self.document(current_emotion)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()

In [3]:
def main():
    bot = SnDBot()
    try:
        bot.run()
    except KeyboardInterrupt:
        print("\nStopping Emotion Detection Bot")
    except Exception as e:
        print(f"Error occured: {str(e)}")
    finally:
        if hasattr(bot, 'cap') and bot.cap is not None:
            bot.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [ ]:
#img = cv2.imread('img2.jpg')
#plt.imshow(img[:,:,: : -1])
#plt.show()

In [ ]:
#result = DeepFace.analyze(img, actions = ['emotion'])
#dominant_emotion = max(result[0]['emotion'].items(), key=lambda x: x[1])
#print(f"{dominant_emotion[0]} {dominant_emotion[1]:.2f}\n")
#print(result[0]['dominant_emotion'])